In [12]:
import sys
import pandas as pd
import json
import csv
import pickle
from pathlib import Path
from bs4 import BeautifulSoup
import json
import multiprocessing
from tqdm import tqdm
import pandas as pd
import math
import multiprocessing as mp
from bs4 import BeautifulSoup
from collections import defaultdict
from tqdm import tqdm


class Languages:
    esga = 'ES-GA'
    si = 'SI'
    gb = 'GB'
    hu = 'HU'
    ua = 'UA'
    all = [esga, si, gb, hu, ua]

In [3]:
with open('artefacts/parlamint_with_embeddings.json', 'r') as f:
    parlamint = json.load(f)
parlamint_es_ga = parlamint['ES-GA']
parlaming_gb = parlamint['GB']
parlamint_hu = parlamint['HU']
parlamint_ua = parlamint['UA']
parlamint_si = parlamint['SI']
parlamints = [
    parlamint_es_ga,
    parlamint_si,
    parlaming_gb,
    parlamint_hu,
    parlamint_ua,
]
artefacts = Path('artefacts/pandas_frames')
artefacts.mkdir(exist_ok=True)

In [ ]:
embedding_files = [
    ('SI', 'ParlaMint_SI_embeddings_truncated.pkl'),
    ('ES-GA', 'ParlaMint_EE_embeddings_truncated.pkl'),
    ('GB', 'ParlaMint_GB_commons_embeddings_truncated.pkl'),
    ('GB', 'ParlaMint_GB_lords_embeddings_truncated.pkl'),
    ('HU', 'ParlaMint_HU_embeddings_truncated.pkl'),
    ('UA', 'ParlaMint_UA_embeddings_truncated.pkl'),
]

embeddings = []
for language, embedding_file in embedding_files:
    with open(Path('artefacts/bojan') / embedding_file, 'rb') as f:
        print(f'Loading', embedding_file)
        embeddings.append((language, pickle.load(f)))

## Create embeddings dataframe

In [14]:
df = pd.DataFrame(columns=['id', 'language', 'embedding'])
for language, lang_embs in embeddings:
    df = pd.concat([df, pd.DataFrame([
        (eid, language, embedding)
        for eid, embedding in lang_embs.items()
    ], columns=['id', 'language', 'embedding'])])
df.reset_index().to_feather(artefacts / 'embeddings.feather')

## Create ParlaMint dataframe

In [6]:
def worker(args):
    language, xml_file = args
    with open(xml_file, 'r') as f:
        soup = BeautifulSoup(f.read(), 'xml')
    sentences = []
    for speech in soup.find_all('u'):
        speech_id = speech['xml:id']
        for sentence in speech.find_all('s'):
            sentence_id = sentence['xml:id']
            sentence_text = ''
            lemmas = {}
            lemmas_list = []
            for unit in sentence.find_all(['w', 'pc']):
                text = unit.get_text()
                if not text:
                    continue
                if unit.name == 'w':
                    sentence_text += ' ' + unit.get_text()
                    lemma = unit.get('lemma')
                    if lemma and lemma.isalpha():
                        lemmas[lemma] = lemmas.get(lemma, 0) + 1
                        lemmas_list.append(lemma)
                elif unit.name == 'pc':
                    sentence_text += unit.get_text()
            sentence_text = sentence_text.strip()
            sentences.append([
                sentence_id,
                speech_id,
                language,
                sentence_text,
                ' '.join(lemmas_list),
            ])
    return sentences


def get_xml_sentences(parlamint):
    all_sentences = []

    with mp.Pool(mp.cpu_count()) as pool:
        parlamint = [(i['language'], i['xml_path']) for i in parlamint]
        for result in tqdm(pool.imap(worker, parlamint), total=len(parlamint)):
            all_sentences.extend(result)

    return all_sentences


for parlamint in parlamints:
    all_sentences = get_xml_sentences(parlamint)
    df = pd.DataFrame(all_sentences, columns=['sentence_id', 'speech_id', 'language', 'text', 'lemmas'])
    df.to_feather(artefacts / f'{parlamint[0]["language"]}_parlamint_sentences.feather')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1091/1091 [01:43<00:00, 10.55it/s]


In [8]:
df

,sentence_id,speech_id,language,text,lemmas
0,ParlaMint-UA_2014-07-03-m1.u1.p1.s1,ParlaMint-UA_2014-07-03-m1.u1,UA,"Шановні народні депутати, прошу сідати на свої...",шановний народний депутат просити сідати на св...
1,ParlaMint-UA_2014-07-03-m1.u1.p1.s2,ParlaMint-UA_2014-07-03-m1.u1,UA,Ми розпочинаємо вечірнє засідання Верховної Ра...,ми розпочинати вечірній засідання верховний ра...
2,ParlaMint-UA_2014-07-03-m1.u1.p2.s1,ParlaMint-UA_2014-07-03-m1.u1,UA,"Я прошу всіх депутатів зайти до сесійної зали,...",я просити весь депутат зайти до сесійний зала ...
3,ParlaMint-UA_2014-07-03-m1.u1.p3.s1,ParlaMint-UA_2014-07-03-m1.u1,UA,"А ми, шановні колеги, давайте почнемо з закону...",а ми шановний колега давати почнути з закон на...
4,ParlaMint-UA_2014-07-03-m1.u1.p3.s2,ParlaMint-UA_2014-07-03-m1.u1,UA,Це проект Закону про реструктуризацію кредитни...,це проект закон про реструктуризація кредитний...
...,...,...,...,...,...
1586343,ParlaMint-UA_2019-04-23-m1.u256.p2.s8,ParlaMint-UA_2019-04-23-m1.u256,UA,"Я наголошую: ніяких відряджень, ніяких відпусток.",я наголошувати ніякий відрядження ніякий відпу...
1586344,ParlaMint-UA_2019-04-23-m1.u256.p2.s9,ParlaMint-UA_2019-04-23-m1.u256,UA,До мене приходило кілька депутатів з відпустка...,до я приходити кілька депутат з відпустка відк...
1586345,ParlaMint-UA_2019-04-23-m1.u256.p2.s10,ParlaMint-UA_2019-04-23-m1.u256,UA,"Відкладіть, хай усі до одного будуть в залі, і...",відкласти хай увесь до один бути в зал і я впе...
1586346,ParlaMint-UA_2019-04-23-m1.u256.p3.s1,ParlaMint-UA_2019-04-23-m1.u256,UA,Зараз вечірнє засідання Верховної Ради України...,зараз вечірній засідання верховний рада Україн...


In [15]:
import pandas as pd
import csv
from pathlib import Path
from multiprocessing import Pool, cpu_count
from tqdm import tqdm


def process_file(metadata):
    with open(metadata, 'r') as file:
        reader = csv.reader(file, delimiter='\t')
        columns = next(reader)
        rows = list(reader)
    return rows, columns


# get list of metadata files
files = list(Path('data/nonannotated').glob('ParlaMint-*.txt/*/ParlaMint-*meta.tsv'))

# set up a multiprocessing pool and process files in parallel
with Pool(cpu_count()) as pool:
    results = list(tqdm(pool.imap(process_file, files), total=len(files)))

# aggregate results
dataframes = [pd.DataFrame(rows, columns=columns) for rows, columns in results]
df = pd.concat(dataframes)

df.reset_index().to_feather(artefacts / 'parlamint_metadata.feather')
df

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 5689/5689 [00:07<00:00, 807.12it/s]


,ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,Subcorpus,Speaker_role,Speaker_MP,Speaker_Minister,Speaker_party,Speaker_party_name,Party_status,Speaker_name,Speaker_gender,Speaker_birth
0,ParlaMint-ES-GA_2017-09-26-DSPG040.u1,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
1,ParlaMint-ES-GA_2017-09-26-DSPG040.u2,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
2,ParlaMint-ES-GA_2017-09-26-DSPG040.u3,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Regular,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Arias Rodríguez, Raquel",F,1966
3,ParlaMint-ES-GA_2017-09-26-DSPG040.u4,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
4,ParlaMint-ES-GA_2017-09-26-DSPG040.u5,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,ParlaMint-UA_2019-11-13-m0.u214,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Regular,MP,-,фЄС,"Фракція політичної партії ""Європейська солідар...",Opposition,"Олексійович Гончаренко, Олексій",M,1980
214,ParlaMint-UA_2019-11-13-m0.u215,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Chairperson,MP,-,,,,"Олександрович Разумков, Дмитро",M,1983
215,ParlaMint-UA_2019-11-13-m0.u216,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Regular,MP,-,фЄС,"Фракція політичної партії ""Європейська солідар...",Opposition,"Володимирівна Геращенко, Ірина",F,1971
216,ParlaMint-UA_2019-11-13-m0.u217,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Chairperson,MP,-,,,,"Олександрович Разумков, Дмитро",M,1983


## Generate speech_id to lemmas mapping

In [13]:
sentences = {}
for language in Languages.all:
    print('loading', language)
    sentences[language] = pd.read_feather(artefacts / f'{language}_parlamint_sentences.feather')

loading ES-GA
loading SI
loading GB
loading HU
loading UA


In [16]:
speeches = {}
for language, sentences_by_language in tqdm(sentences.items()):
    speeches[language] = {}
    for speech_id, sentences in sentences_by_language.groupby('speech_id'):
        speeches[language][speech_id] = []
        for lemmas in sentences.lemmas:
            speeches[language][speech_id].append(lemmas)
        speeches[language][speech_id] = ' '.join(speeches[language][speech_id])

for language, mapping in speeches.items():
    with open(artefacts / f'speech2lemmas_{language}.pkl', 'wb') as f:
        pickle.dump(mapping, f)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:43<00:00,  8.62s/it]


In [9]:
speeches = {}
for language in tqdm(sentences):
    with open(artefacts / f'speech2lemmas_{language}.pkl', 'rb') as f:
        speeches[language] = pickle.load(f)

NameError: name 'sentences' is not defined

## Generate corpuses based on list of keywords

In [44]:
def get_speech_ids(speech2lemmas: dict[str, set[str]], keywords: set[str]):
    for speech_id, lemmas in speech2lemmas.items():
        lemmas = lemmas.lower()
        for keyword in keywords:
            if keyword in lemmas:
                yield speech_id
                break


def get_corpuses(topic: set[str]):
    corpuses = {}
    for language, speech2lemmas in speeches.items():
        speech_ids = list(get_speech_ids(speech2lemmas, topic))
        corpuses[language] = speech_ids
    return corpuses


def print_corpuses(corpuses):
    for lang, corpus in corpuses.items():
        print(f'{lang}:', len(corpus))
        print(corpus[:10])
        print('-' * 40)


def save_corpuses(topic, corpuses):
    artefacts_dir = artefacts / 'by_topic'
    artefacts_dir.mkdir(exist_ok=True)
    df = pd.DataFrame.from_dict({
        'common': set([t for l in corpuses.values() for t in l]),
        **corpuses
    }, orient='index').transpose()
    df.to_feather(artefacts_dir / f'{topic}.feather')
    return df


In [46]:
save_corpuses('eu', get_corpuses({'eu'}))

,common,ES-GA,SI,GB,HU,UA
0,ParlaMint-GB_2016-07-05-lords.u126,ParlaMint-ES-GA_2015-01-26-DSPGDP007.u10,ParlaMint-SI_2000-11-16-SDZ3-Izredna-01.ana.u35,ParlaMint-GB_2015-01-05-commons.u238,u2014-05-06-0,ParlaMint-UA_2013-06-05-m0.u58
1,ParlaMint-GB_2018-10-22-commons.u452,ParlaMint-ES-GA_2015-01-26-DSPGDP007.u12,ParlaMint-SI_2000-11-17-SDZ3-Izredna-01.ana.u20,ParlaMint-GB_2015-01-05-commons.u256,u2014-05-06-1,ParlaMint-UA_2013-10-08-m0.u49
2,ParlaMint-GB_2019-10-19-commons.u52,ParlaMint-ES-GA_2015-01-26-DSPGDP007.u18,ParlaMint-SI_2000-11-17-SDZ3-Izredna-01.ana.u3,ParlaMint-GB_2015-01-05-commons.u263,u2014-05-10-0,ParlaMint-UA_2013-10-08-m0.u69
3,u2016-11-08-45,ParlaMint-ES-GA_2015-01-26-DSPGDP007.u2,ParlaMint-SI_2000-11-21-SDZ3-Izredna-01.ana.u2,ParlaMint-GB_2015-01-05-commons.u264,u2014-05-10-3,ParlaMint-UA_2015-03-05-m0.u2
4,ParlaMint-GB_2019-10-15-commons.u26,ParlaMint-ES-GA_2015-01-26-DSPGDP007.u22,ParlaMint-SI_2000-11-30-SDZ3-Izredna-02.ana.u102,ParlaMint-GB_2015-01-05-commons.u351,u2014-05-12-1,ParlaMint-UA_2015-12-08-m0.u56
...,...,...,...,...,...,...
165870,ParlaMint-SI_2021-12-13-SDZ8-Redna-28.ana.u242,None,None,None,None,None
165871,ParlaMint-GB_2015-01-22-commons.u351,None,None,None,None,None
165872,ParlaMint-GB_2018-11-28-commons.u157,None,None,None,None,None
165873,ParlaMint-SI_2016-10-25-SDZ7-Redna-23.ana.u76,None,None,None,None,None
